In [1]:
import numpy as np
import tensorflow as tf
import scipy.io
import tensorly as ts
import os
os.chdir("G:/jupyter/Guangzhou-data-set")
tensor = scipy.io.loadmat('tensor.mat')
tensor = tensor['tensor']
# random_matrix = scipy.io.loadmat('Guangzhou-data-set/random_matrix.mat')
# random_matrix = random_matrix['random_matrix']
# random_tensor = scipy.io.loadmat('Guangzhou-data-set/random_tensor.mat')
# random_tensor = random_tensor['random_tensor']

tensor = np.array(tensor)
print(tensor.shape)
tensor = tensor.reshape(214,-1)
print(tensor.shape)

(214, 61, 144)
(214, 8784)


Using numpy backend.


In [2]:
missing_rate = 0.1
condition = np.random.binomial(1, 1-missing_rate, np.shape(tensor)[0]*np.shape(tensor)[1])
condition = np.reshape(condition,(np.shape(tensor)[0],np.shape(tensor)[1]))
mask = np.where(condition<1,0.,1.)
print(mask.shape)
sparse_tensor = np.multiply(tensor, mask)
print(sparse_tensor.shape)

mean = (np.sum(sparse_tensor)*1.0/np.sum(mask))
init_tensor = sparse_tensor
init_tensor[init_tensor==0] = mean
print(init_tensor.shape)

(214, 8784)
(214, 8784)
(214, 8784)


In [3]:
dim = tensor.shape
rank1 = 150
rank2 = 8000
tf.reset_default_graph()

init_tensor_holder = tf.placeholder(tf.float32, shape=[dim[0], dim[1]]) 
sparse_tensor_holder = tf.placeholder(tf.float32, shape=[dim[0], dim[1]])    
tensor_holder = tf.placeholder(tf.float32, shape=[dim[0], dim[1]])  
mask_tensor_holder =  tf.placeholder(tf.float32, shape=[dim[0], dim[1]])

sess = tf.Session()

su, uu, vu = tf.svd(init_tensor_holder)

uu = uu[:,0:rank1]
vu = tf.transpose(vu[:,0:rank1])

# initialzation construction
U_,V_=sess.run([uu,vu], feed_dict={init_tensor_holder:init_tensor})

U = tf.Variable(U_)
V = tf.Variable(V_)


Train_Param=[U,V]

impute_tensor = tf.matmul(U, V)

In [12]:
# loss function

loss = tf.reduce_mean((mask_tensor_holder * tensor_holder - mask_tensor_holder * impute_tensor) ** 2)
loss_evaluation = tf.reduce_sum(( (1.-mask_tensor_holder)*tensor_holder - (1.-mask_tensor_holder) * impute_tensor)  ** 2) / (tf.math.reduce_sum(1.-mask_tensor_holder))
loss_all = tf.reduce_mean((tensor_holder - impute_tensor) ** 2)
# r_g = tf.contrib.layers.l1_regularizer(scale=0.00001, scope=None)
# weights = tf.trainable_variables()
# r1 = tf.contrib.layers.apply_regularization(r_g, weights)
loss_reg = loss

solver = tf.train.AdamOptimizer(0.02).minimize(loss_reg, var_list=Train_Param)

In [13]:
sess.run(tf.global_variables_initializer())

for i in range(50000):
    _, loss_reg_,it = sess.run([solver, loss_reg, impute_tensor], feed_dict = 
                           {mask_tensor_holder: mask, sparse_tensor_holder: sparse_tensor, 
                            tensor_holder:tensor})
    if i % 100==0:
        imputation_result,loss_eval,loss_train,loss_all_ = sess.run([impute_tensor,loss_evaluation,loss,loss_all], feed_dict =
                                     {mask_tensor_holder: mask, sparse_tensor_holder: sparse_tensor, 
                                      tensor_holder:tensor})
        
        print('epoch = {:}'.format(i))
        print('TRAIN RMSE = {:.6}'.format(np.sqrt(loss_train)))
        print('TEST RMSE = {:.6}'.format(np.sqrt(loss_eval)))
        print('Overall RMSE = {:.6}'.format(np.sqrt(loss_all_)))
        print('-------------------------------------------------')

        

epoch = 0
TRAIN RMSE = 38.1451
TEST RMSE = 40.2674
Overall RMSE = 40.2154
-------------------------------------------------
epoch = 100
TRAIN RMSE = 5.81588
TEST RMSE = 6.19511
Overall RMSE = 6.13707
-------------------------------------------------
epoch = 200
TRAIN RMSE = 4.80489
TEST RMSE = 5.29221
Overall RMSE = 5.08809
-------------------------------------------------
epoch = 300
TRAIN RMSE = 3.98956
TEST RMSE = 4.57824
Overall RMSE = 4.24421
-------------------------------------------------
epoch = 400
TRAIN RMSE = 3.40926
TEST RMSE = 4.29365
Overall RMSE = 3.66977
-------------------------------------------------
epoch = 500
TRAIN RMSE = 2.78292
TEST RMSE = 4.14201
Overall RMSE = 3.07586
-------------------------------------------------
epoch = 600
TRAIN RMSE = 2.17008
TEST RMSE = 4.07568
Overall RMSE = 2.52408
-------------------------------------------------
epoch = 700
TRAIN RMSE = 1.69862
TEST RMSE = 4.06116
Overall RMSE = 2.12962
--------------------------------------------

KeyboardInterrupt: 

In [12]:
su, uu, vu = tf.svd(init_tensor_holder)
print(su)
print(uu)
print(vu)

Tensor("Svd_2:0", shape=(214,), dtype=float32)
Tensor("Svd_2:1", shape=(214, 214), dtype=float32)
Tensor("Svd_2:2", shape=(8784, 214), dtype=float32)
